In [11]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from itertools import combinations 
from numpy import savetxt
import random

In [12]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import PolynomialFeatures
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout, Input
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras

In [30]:
input_df = pd.read_csv("data/input_data_nn.csv", sep=";")
output_df = pd.read_csv("data/output_data_nn.csv", sep=";")
df = pd.concat([input_df, output_df], axis=1)
df = df.loc[~(df==0).all(axis=1)].reset_index()

In [24]:
output_df.describe()

,tensao_barramento11,tensao_barramento12,tensao_barramento24,pot_ativa_inj_barramento11,pot_ativa_inj_barramento12,pot_ativa_inj_barramento24,pot_reativa_inj_barramento11,pot_reativa_inj_barramento12,pot_reativa_inj_barramento24
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000
mean,0.996569,0.997564,0.993244,44.794038,65.017462,98.531922,-15.602551,-3.119726,25.755603
std,0.021639,0.021008,0.021868,32.318890,46.834371,53.707492,11.060453,15.983727,10.808798
min,0.952364,0.953941,0.950000,-27.531718,-45.061452,-29.154621,-34.601205,-30.758948,-0.197103
25%,0.986579,0.989016,0.981990,17.824650,35.544467,54.846269,-23.606006,-15.992032,19.179831
50%,1.005131,1.006796,1.001604,43.524182,61.976083,95.024840,-17.984876,-9.813467,26.085991
75%,1.012191,1.012873,1.011476,66.443556,94.266833,134.378609,-7.401176,8.937662,33.428328
max,1.034885,1.032754,1.022086,159.904392,233.978721,261.648979,16.029554,47.883536,57.932657


In [25]:
output_df.describe()

,tensao_barramento11,tensao_barramento12,tensao_barramento24,pot_ativa_inj_barramento11,pot_ativa_inj_barramento12,pot_ativa_inj_barramento24,pot_reativa_inj_barramento11,pot_reativa_inj_barramento12,pot_reativa_inj_barramento24
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000
mean,0.996569,0.997564,0.993244,44.794038,65.017462,98.531922,-15.602551,-3.119726,25.755603
std,0.021639,0.021008,0.021868,32.318890,46.834371,53.707492,11.060453,15.983727,10.808798
min,0.952364,0.953941,0.950000,-27.531718,-45.061452,-29.154621,-34.601205,-30.758948,-0.197103
25%,0.986579,0.989016,0.981990,17.824650,35.544467,54.846269,-23.606006,-15.992032,19.179831
50%,1.005131,1.006796,1.001604,43.524182,61.976083,95.024840,-17.984876,-9.813467,26.085991
75%,1.012191,1.012873,1.011476,66.443556,94.266833,134.378609,-7.401176,8.937662,33.428328
max,1.034885,1.032754,1.022086,159.904392,233.978721,261.648979,16.029554,47.883536,57.932657


In [26]:
def augment(x, y, augmenter = None):
    x, augmenter = _add_interactions(x, augmenter)
    return x, y, augmenter, x.columns

def _add_interactions(df, augmenter):
    combos = list(combinations(list(df.columns), 2))
    colnames = list(df.columns) + ['_'.join(x) for x in combos]

    if augmenter:
        df = augmenter.transform(df)
    else:
        augmenter = PolynomialFeatures(interaction_only=True, include_bias=False)
        df = augmenter.fit_transform(df)

    df = pd.DataFrame(df)
    df.columns = colnames
    
    noint_indicies = [i for i, x in enumerate(list((df == 0).all())) if x]
    df = df.drop(df.columns[noint_indicies], axis=1)
    
    return df, augmenter

In [31]:
def run_main(df, idx, idx2, ids_to_pred):    
    cols = [
        'carga_subrede_138kv',
        'pot_eolica_subrede_138kv',
        'pot_solar_subrede_138kv',
        'carga_subrede_230kv',
        'pot_eolica_subrede_230kv',
        'pot_solar_subrede_230kv'
    ]
    
    cols_output = [
        'tensao_barramento11', 'tensao_barramento12', 'tensao_barramento24',
        'pot_ativa_inj_barramento11', 'pot_ativa_inj_barramento12', 'pot_ativa_inj_barramento24',
        'pot_reativa_inj_barramento11', 'pot_reativa_inj_barramento12', 'pot_reativa_inj_barramento24'
    ]
    predicted_df = pd.DataFrame(columns=cols_output, index=ids_to_pred)
    new_df = df.copy()

    cols_pred = ['tensao_barramento11']
    model_id = 'tensao_11'
    tensao_11_df, predicted_11 = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred, update_test=True)

    cols_pred = ['tensao_barramento12']
    model_id = 'tensao_12'
    tensao_12_df, predicted_12 = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred, update_test=True)

    cols_pred = ['tensao_barramento24']
    model_id = 'tensao_24'
    tensao_24_df, predicted_24 = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred, update_test=True)
    
    new_df['tensao_barramento11'] = tensao_11_df['tensao_barramento11']
    new_df['tensao_barramento12'] = tensao_12_df['tensao_barramento12']
    new_df['tensao_barramento24'] = tensao_24_df['tensao_barramento24']
    
    predicted_df['tensao_barramento11'] = predicted_11
    predicted_df['tensao_barramento12'] = predicted_12
    predicted_df['tensao_barramento24'] = predicted_24
    
    cols += ['tensao_barramento11', 'tensao_barramento12', 'tensao_barramento24']
    
    cols_pred = ['pot_ativa_inj_barramento11']
    model_id = 'pot_ativa_inj_barramento11'
    result_df, predictions = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred)
    predicted_df[model_id] = predictions

    cols_pred = ['pot_ativa_inj_barramento12']
    model_id = 'pot_ativa_inj_barramento12'
    result_df, predictions = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred)
    predicted_df[model_id] = predictions

    cols_pred = ['pot_ativa_inj_barramento24']
    model_id = 'pot_ativa_inj_barramento24'
    result_df, predictions = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred)
    predicted_df[model_id] = predictions
    
    cols_pred = ['pot_reativa_inj_barramento11']
    model_id = 'pot_reativa_inj_barramento11'
    result_df, predictions = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred)
    predicted_df[model_id] = predictions
    
    cols_pred = ['pot_reativa_inj_barramento12']
    model_id = 'pot_reativa_inj_barramento12'
    result_df, predictions = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred)
    predicted_df[model_id] = predictions

    cols_pred = ['pot_reativa_inj_barramento24']
    model_id = 'pot_reativa_inj_barramento24'
    result_df, predictions = run(new_df, cols, cols_pred, model_id, idx, ids_to_pred)
    predicted_df[model_id] = predictions    
    
    predicted_df.to_csv(f'new-{idx}-{idx2}.csv')

    return new_df

In [32]:
def run(df, cols, cols_pred, model_id, idx, ids_to_pred, update_test = False):    
    _df = df[cols + cols_pred]
    
    X, Y = _df[cols], _df[cols_pred]
    X, Y, _, x_cols, = augment(X, Y)
    _df_augm = pd.concat([X, Y],  axis=1)

    df_for_testing = _df_augm.loc[ids_to_pred]
    df_for_training = _df_augm.loc[~_df_augm.index.isin(ids_to_pred)]
    
    df_for_training = df_for_training.sample(frac=1)
    
    trainX, trainY = df_for_training.drop(cols_pred, axis=1), df_for_training[cols_pred]
    testX, testY = df_for_testing.drop(cols_pred, axis=1), df_for_testing[cols_pred]
    
    scaler = StandardScaler()
    trainX = scaler.fit_transform(trainX)
    testX = scaler.transform(testX)
    
    print(f" ========== MODEL {model_id} {idx} ============")
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)
    mc = ModelCheckpoint(f"models-new/ann-{model_id}-{idx}.h5", monitor='val_loss', mode='min', save_best_only=True, verbose=0)

    model = Sequential()
    model.add(Input(shape=(trainX.shape[1],)))
    model.add(Dense(trainX.shape[-1] ** 2, activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(trainX.shape[-1], activation = 'relu')) 
    model.add(Dropout(0.2))
    model.add(Dense(len(cols_pred)))

    model.compile(loss = 'mae', optimizer = 'adam')
    model.fit(
        trainX, trainY,
        batch_size = 10, epochs = 20,
        callbacks = [es, mc],
        validation_data = (testX, testY),
        verbose=0
    )

    _predictions = model.predict(testX)    
    rmse = sqrt(mean_squared_error(testY, _predictions.flatten()))
    print(f"RMSE: {rmse}")
        
    results.append(rmse)
    
    if update_test:
        df_for_testing[cols_pred] = _predictions

    return pd.concat([df_for_training, df_for_testing]), _predictions  

In [29]:
indexes = np.array(df.index.tolist())
random.shuffle(indexes)
ids_to_pred = np.split(indexes, 3)

results_all = []
for i in range(1):
    results_per_ids = []
    for idx2 in range(len(ids_to_pred)):
        results = []
        run_main(df, i, idx2, ids_to_pred[idx2])
        results_per_ids.append(results)
        
    results_all.append(results_per_ids)

 ========== MODEL tensao_11 0 ============
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When 

KeyboardInterrupt: 

In [21]:
for _result in results_all:
    print(results_all.index(_result))
    for fold in _result:
        print(f"{_result.index(fold)}: {sum(fold)}")

0
0: 32.33497086128333
1: 36.889005154036376
2: 36.68749314981343
3: 31.932388827029182
1
0: 32.26412368037137
1: 31.990872924834257
2: 36.0298650015969
3: 32.30431549681663
2
0: 31.72844917901247
1: 29.84034210963254
2: 34.37873809844469
3: 38.184718100379435
3
0: 34.15181711482428
1: 30.558933149246954
2: 30.1250874789621
3: 31.1846092629998
4
0: 34.86386086977219
1: 28.93438134306144
2: 36.44611309999486
3: 33.4754096497843
5
0: 37.116041669193386
1: 37.296140359486756
2: 30.919280629845613
3: 39.44656374003831
6
0: 31.38229179727558
1: 30.653259296110438
2: 31.444674673125057
3: 36.76260480496647
7
0: 28.349547201857266
1: 33.73417886152881
2: 39.737148167795624
3: 37.59890026817541
8
0: 29.559890665146096
1: 40.83094120187875
2: 26.866222288396877
3: 37.813916555230335
9
0: 38.21746676037965
1: 37.003628372655385
2: 47.06004230959253
3: 34.13258223262406


In [26]:
print(f"Fold 0: {results_all[7][0]}")
print(f"Fold 1: {results_all[4][1]}")
print(f"Fold 2: {results_all[8][2]}")
print(f"Fold 3: {results_all[3][3]}")

Fold 0: [0.005618806104382909, 0.006374496128408189, 0.005003273984089787, 3.4735610445018055, 5.057355698378199, 3.6107973146928116, 7.466659649968084, 6.293739685672916, 2.4304372324265673]
Fold 1: [0.004912833436191945, 0.006315874222206748, 0.0050400543592956865, 5.882427740243566, 3.5072674248462787, 3.786373119903367, 6.751693446504084, 6.168161926473203, 2.822188923073245]
Fold 2: [0.004500589408310584, 0.004620877592182861, 0.005015842571451259, 3.953451113791048, 4.887246335988537, 4.617532544758241, 5.595597117037946, 5.241330146546544, 2.5569277207026158]
Fold 3: [0.005904257261356303, 0.004717165985727671, 0.005687405161361025, 3.2969980512393264, 5.985023419883928, 3.5217441168282497, 8.310837484354026, 6.692027891640731, 3.3616694706450936]


In [32]:
df_7_0 = pd.read_csv('7-0.csv')
df_7_0 = df_7_0.rename(columns={"Unnamed: 0": "id"})

df_4_1 = pd.read_csv('4-1.csv')
df_4_1 = df_4_1.rename(columns={"Unnamed: 0": "id"})

df_8_2 = pd.read_csv('8-2.csv')
df_8_2 = df_8_2.rename(columns={"Unnamed: 0": "id"})

df_3_3 = pd.read_csv('3-3.csv')
df_3_3 = df_3_3.rename(columns={"Unnamed: 0": "id"})

In [36]:
df_best_predicted = pd.concat([df_7_0, df_4_1, df_8_2, df_3_3])
df_best_predicted = df_best_predicted.sort_values(by=['id'])

In [41]:
df_best_predicted.columns

Index(['id', 'tensao_barramento11', 'tensao_barramento12',
       'tensao_barramento24', 'pot_ativa_inj_barramento11',
       'pot_ativa_inj_barramento12', 'pot_ativa_inj_barramento24',
       'pot_reativa_inj_barramento11', 'pot_reativa_inj_barramento12',
       'pot_reativa_inj_barramento24'],
      dtype='object')

In [48]:
df['pot_reativa_inj_barramento24']

0       27.959498
1       23.210810
2       19.182219
3       16.913922
4       15.967630
          ...    
8755    14.528968
8756    33.922939
8757    27.785688
8758    24.356531
8759    21.964095
Name: pot_reativa_inj_barramento24, Length: 8760, dtype: float64

In [45]:
 df_best_predicted['pot_reativa_inj_barramento24']

1296    27.959498
1339    23.210810
1886    19.182219
2123    16.913922
502     15.967630
          ...    
139     14.528968
154     33.922939
704     27.785688
1381    24.356531
2078    21.964095
Name: pot_reativa_inj_barramento24, Length: 8760, dtype: float64

In [40]:
print(f"RMSE tensao_barramento11: {sqrt(mean_squared_error(df['tensao_barramento11'], df_best_predicted['tensao_barramento11']))}")
print(f"RMSE tensao_barramento12: {sqrt(mean_squared_error(df['tensao_barramento12'], df_best_predicted['tensao_barramento12']))}")
print(f"RMSE tensao_barramento24: {sqrt(mean_squared_error(df['tensao_barramento24'], df_best_predicted['tensao_barramento24']))}")

print(f"RMSE pot_ativa_inj_barramento11: {sqrt(mean_squared_error(df['pot_ativa_inj_barramento11'], df_best_predicted['pot_ativa_inj_barramento11']))}")
print(f"RMSE pot_ativa_inj_barramento12: {sqrt(mean_squared_error(df['pot_ativa_inj_barramento12'], df_best_predicted['pot_ativa_inj_barramento12']))}")
print(f"RMSE pot_ativa_inj_barramento24: {sqrt(mean_squared_error(df['pot_ativa_inj_barramento24'], df_best_predicted['pot_ativa_inj_barramento24']))}")


RMSE tensao_barramento11: 0.005263612667191903
RMSE tensao_barramento12: 0.005570651504552013
RMSE tensao_barramento24: 0.005194712520332189
RMSE pot_ativa_inj_barramento11: 0.0
RMSE pot_ativa_inj_barramento12: 0.0
RMSE pot_ativa_inj_barramento24: 0.0


In [49]:
df.loc[[924]]

,carga_subrede_138kv,pot_eolica_subrede_138kv,pot_solar_subrede_138kv,carga_subrede_230kv,pot_eolica_subrede_230kv,pot_solar_subrede_230kv,status_1_gerador_subrede138kv,status_2_gerador_subrede138kv,status_3_gerador_subrede138kv,status_4_gerador_subrede138kv,...,status_21_linha_subrede_230kv,tensao_barramento11,tensao_barramento12,tensao_barramento24,pot_ativa_inj_barramento11,pot_ativa_inj_barramento12,pot_ativa_inj_barramento24,pot_reativa_inj_barramento11,pot_reativa_inj_barramento12,pot_reativa_inj_barramento24
924,1007.809524,401.818085,470.954682,1261.000862,71.944557,121.463748,1,1,1,1,...,1,0.981256,0.984862,0.975793,-58.51831,8.187012,24.743374,2.695873,8.599397,24.234942


In [49]:
results = {}
for i in range(30):
    run_main(df, i)

 ========== MODEL tensao_11 0 ============

Epoch 1: val_loss improved from inf to 0.09023, saving model to models-new/ann-tensao_11-0.h5

Epoch 2: val_loss improved from 0.09023 to 0.05260, saving model to models-new/ann-tensao_11-0.h5

Epoch 3: val_loss improved from 0.05260 to 0.02550, saving model to models-new/ann-tensao_11-0.h5

Epoch 4: val_loss improved from 0.02550 to 0.01142, saving model to models-new/ann-tensao_11-0.h5

Epoch 5: val_loss improved from 0.01142 to 0.00536, saving model to models-new/ann-tensao_11-0.h5
83/83 [==============================] - 0s 962us/step
> /tmp/ipykernel_7672/1406847328.py(42)run()
     40     _predictions = model.predict(testX)
     41     import pdb; pdb.set_trace()
---> 42     rmse = sqrt(mean_squared_error(testY, _predictions))
     43     print(f"RMSE: {rmse}")
     44 

ipdb> _predictions
array([[1.0164385],
       [1.0171256],
       [0.9599639],
       ...,
       [1.0027887],
       [1.0020775],
       [1.0048659]], dtype=float32)
i

BdbQuit: 

In [41]:
np_results = np.array(list(results.values()))

In [42]:
print("Without augmentation and new dataset (MAE)")
for i in range(9):
    print(sum(np_results[:, i]) / 10)

Without augmentation and new dataset (MAE)
0.0070890919168230425
0.007541286872679962
0.007523444369988579
10.894243509357642
9.267402284153766
9.371384044918205
11.252243935611665
9.437299328059101
3.8338009920329936


In [38]:
print("With augmentation and new dataset (MAE)")
for i in range(9):
    print(sum(np_results[:, i]) / 10)

With augmentation and new dataset (MAE)
0.005665393469191508
0.005968114200631143
0.006086502990279516
4.306654861961782
4.929694810852576
5.969859277039574
8.037082282762213
6.949571854237709
3.0683251384838686


In [33]:
print("With augmentation and new dataset with 138kv")
for i in range(9):
    print(sum(np_results[:, i]) / 10)

With augmentation and new dataset with 138kv
0.0066337208966503495
0.008000043449722296
0.007588173150638114
4.3975292615658095
6.61129412958843
7.356855111909887
13.70290618834233
11.36563117387233
3.3401618094366077


In [27]:
print("With augmentation and new dataset")
for i in range(9):
    print(sum(np_results[:, i]) / 10)

With augmentation and new dataset
0.006926622726284169
0.007056876000442106
0.0065804778545112365
5.432553705919982
5.59374025258896
5.02091411051572
10.912814636247882
9.373484699735132
3.2685546341750764


In [26]:
print("Without augmentation and new dataset")
for i in range(9):
    print(sum(np_results[:, i]) / 10)

Without augmentation and new dataset
0.006926622726284169
0.007056876000442106
0.0065804778545112365
5.432553705919982
5.59374025258896
5.02091411051572
10.912814636247882
9.373484699735132
3.2685546341750764


In [20]:
for i in range(9):
    print(sum(np_results[:, i]) / 10)

0.008563264243237792
0.008851690008478937
0.008759378146553693
3.43743832269994
4.166979929936218
6.164294312250694
5.174641738938345
5.538004377575238
3.710841119860305


In [10]:
for i in range(9):
    print(sum(np_results[:, i]) / 10)

0.0177697726419364
0.017474690752547213
0.01802198466043586
37.83911595853088
47.1200942223504
55.18394663436637
12.37940462639021
16.0528277028151
9.767292352196808


In [14]:
for i in range(9):
    print(sum(np_results[:, i]) / 10)

0.010302829257462714
0.009528249833838341
0.009607028976536487
2.7364607725701724
6.54437777406716
11.961217698318539
6.324785552014249
6.815439932590181
4.192799714407331


In [24]:
max(df['pot_reativa_inj_barramento24']) - min(df['pot_reativa_inj_barramento24'])

58.12975947284735

In [25]:
max(df['pot_reativa_inj_barramento12']) - min(df['pot_reativa_inj_barramento12'])

78.642483761916

In [26]:
max(df['pot_reativa_inj_barramento11']) - min(df['pot_reativa_inj_barramento11'])

50.6307587013866

In [277]:
df = pd.concat([input_df, output_df], axis=1)

In [121]:
from sklearn import preprocessing, model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit

In [39]:
def plot(data):
    figure(figsize = (15, 6), dpi = 80)
    plt.plot(data, label = 'Real')
    #plt.plot(pred, color = 'blue', label = 'Predicted')
    plt.legend()
    plt.show()

In [12]:
cols = ['carga_subrede_138kv',
        'pot_eolica_subrede_138kv',
        'pot_solar_subrede_138kv',
        'carga_subrede_230kv',
        'pot_eolica_subrede_230kv',
        'pot_solar_subrede_230kv']

cols_pred = ['tensao_barramento12']
model_id = 'tensao_12'
tensao_12_df = run(df, cols, cols_pred, model_id, update_test=True)

cols_pred = ['tensao_barramento11']
model_id = 'tensao_11'
tensao_11_df = run(df, cols, cols_pred, model_id, update_test=True)

cols_pred = ['tensao_barramento24']
model_id = 'tensao_24'
tensao_24_df = run(df, cols, cols_pred, model_id, update_test=True)

 ========== MODEL tensao_12 ============


2022-08-19 11:38:01.824417: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-19 11:38:01.824457: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-19 11:38:01.824484: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (schlickmann): /proc/driver/nvidia/version does not exist
2022-08-19 11:38:01.825490: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/20
613/614 [============================>.] - ETA: 0s - loss: 0.2858
Epoch 1: val_loss improved from inf to 0.17861, saving model to models/ann-tensao_12.h5
614/614 [==============================] - 25s 40ms/step - loss: 0.2857 - val_loss: 0.1786
Epoch 2/20
613/614 [============================>.] - ETA: 0s - loss: 0.1822
Epoch 2: val_loss did not improve from 0.17861
614/614 [==============================] - 24s 39ms/step - loss: 0.1822 - val_loss: 0.1906
Epoch 3/20
613/614 [============================>.] - ETA: 0s - loss: 0.1655
Epoch 3: val_loss improved from 0.17861 to 0.13944, saving model to models/ann-tensao_12.h5
614/614 [==============================] - 24s 40ms/step - loss: 0.1654 - val_loss: 0.1394
Epoch 4/20
614/614 [==============================] - ETA: 0s - loss: 0.1540
Epoch 4: val_loss improved from 0.13944 to 0.13924, saving model to models/ann-tensao_12.h5
614/614 [==============================] - 21s 34ms/step - loss: 0.1540 - val_loss: 0.1392
Epoch 5/2

/tmp/ipykernel_66249/2835460861.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_testing[cols_pred[0]] = predictions[:,-1]


 ========== MODEL tensao_11 ============
Epoch 1/20
614/614 [==============================] - ETA: 0s - loss: 0.2649
Epoch 1: val_loss improved from inf to 0.18844, saving model to models/ann-tensao_11.h5
614/614 [==============================] - 25s 39ms/step - loss: 0.2649 - val_loss: 0.1884
Epoch 2/20
613/614 [============================>.] - ETA: 0s - loss: 0.1875
Epoch 2: val_loss improved from 0.18844 to 0.18284, saving model to models/ann-tensao_11.h5
614/614 [==============================] - 21s 34ms/step - loss: 0.1874 - val_loss: 0.1828
Epoch 3/20
614/614 [==============================] - ETA: 0s - loss: 0.1544
Epoch 3: val_loss improved from 0.18284 to 0.16195, saving model to models/ann-tensao_11.h5
614/614 [==============================] - 21s 35ms/step - loss: 0.1544 - val_loss: 0.1620
Epoch 4/20
613/614 [============================>.] - ETA: 0s - loss: 0.1481
Epoch 4: val_loss improved from 0.16195 to 0.10331, saving model to models/ann-tensao_11.h5
614/614 [=====

/tmp/ipykernel_66249/2835460861.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_testing[cols_pred[0]] = predictions[:,-1]


614/614 [==============================] - ETA: 0s - loss: 0.2799
Epoch 1: val_loss improved from inf to 0.13066, saving model to models/ann-tensao_24.h5
614/614 [==============================] - 20s 32ms/step - loss: 0.2799 - val_loss: 0.1307
Epoch 2/20
614/614 [==============================] - ETA: 0s - loss: 0.1675
Epoch 2: val_loss improved from 0.13066 to 0.12800, saving model to models/ann-tensao_24.h5
614/614 [==============================] - 21s 35ms/step - loss: 0.1675 - val_loss: 0.1280
Epoch 3/20
613/614 [============================>.] - ETA: 0s - loss: 0.1388
Epoch 3: val_loss improved from 0.12800 to 0.10169, saving model to models/ann-tensao_24.h5
614/614 [==============================] - 19s 30ms/step - loss: 0.1387 - val_loss: 0.1017
Epoch 4/20
614/614 [==============================] - ETA: 0s - loss: 0.1279
Epoch 4: val_loss did not improve from 0.10169
614/614 [==============================] - 20s 33ms/step - loss: 0.1279 - val_loss: 0.1166
Epoch 5/20
614/614 [

/tmp/ipykernel_66249/2835460861.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_testing[cols_pred[0]] = predictions[:,-1]


In [13]:
df['tensao_barramento11'] = tensao_11_df['tensao_barramento11']
df['tensao_barramento12'] = tensao_12_df['tensao_barramento12']
df['tensao_barramento24'] = tensao_24_df['tensao_barramento24']

In [36]:
cols = ['carga_subrede_138kv',
        'pot_eolica_subrede_138kv',
        'pot_solar_subrede_138kv',
        'carga_subrede_230kv',
        'pot_eolica_subrede_230kv',
        'pot_solar_subrede_230kv',
        'tensao_barramento11',
        'tensao_barramento12',
         'tensao_barramento24'
       ]

In [16]:
cols_pred = ['pot_ativa_inj_barramento11']
model_id = 'pot_ativa_inj_barramento11'
pot_ativa_inj_barramento11_df = run(df, cols, cols_pred, model_id)

 ========== MODEL pot_ativa_inj_barramento11 ============
Epoch 1/20
600/614 [============================>.] - ETA: 0s - loss: 0.0813
Epoch 1: val_loss improved from inf to 0.00606, saving model to models-new/ann-pot_ativa_inj_barramento11.h5
614/614 [==============================] - 3s 4ms/step - loss: 0.0807 - val_loss: 0.0061
Epoch 2/20
603/614 [============================>.] - ETA: 0s - loss: 0.0442
Epoch 2: val_loss improved from 0.00606 to 0.00562, saving model to models-new/ann-pot_ativa_inj_barramento11.h5
614/614 [==============================] - 2s 4ms/step - loss: 0.0441 - val_loss: 0.0056
Epoch 3/20
600/614 [============================>.] - ETA: 0s - loss: 0.0447
Epoch 3: val_loss improved from 0.00562 to 0.00510, saving model to models-new/ann-pot_ativa_inj_barramento11.h5
614/614 [==============================] - 2s 4ms/step - loss: 0.0446 - val_loss: 0.0051
Epoch 4/20
608/614 [============================>.] - ETA: 0s - loss: 0.0421
Epoch 4: val_loss did not improv

In [17]:
cols_pred = ['pot_ativa_inj_barramento12']
model_id = 'pot_ativa_inj_barramento12'
pot_ativa_inj_barramento12_df = run(df, cols, cols_pred, model_id)

 ========== MODEL pot_ativa_inj_barramento12 ============
Epoch 1/20
614/614 [==============================] - ETA: 0s - loss: 0.0757
Epoch 1: val_loss improved from inf to 0.02582, saving model to models-new/ann-pot_ativa_inj_barramento12.h5
614/614 [==============================] - 3s 4ms/step - loss: 0.0757 - val_loss: 0.0258
Epoch 2/20
601/614 [============================>.] - ETA: 0s - loss: 0.0488
Epoch 2: val_loss improved from 0.02582 to 0.02114, saving model to models-new/ann-pot_ativa_inj_barramento12.h5
614/614 [==============================] - 2s 4ms/step - loss: 0.0488 - val_loss: 0.0211
Epoch 3/20
608/614 [============================>.] - ETA: 0s - loss: 0.0455
Epoch 3: val_loss improved from 0.02114 to 0.01111, saving model to models-new/ann-pot_ativa_inj_barramento12.h5
614/614 [==============================] - 2s 4ms/step - loss: 0.0454 - val_loss: 0.0111
Epoch 4/20
609/614 [============================>.] - ETA: 0s - loss: 0.0453
Epoch 4: val_loss improved from 

In [37]:
cols_pred = ['pot_ativa_inj_barramento24']
model_id = 'pot_ativa_inj_barramento24'
pot_ativa_inj_barramento24_df = run(df, cols, cols_pred, model_id)

 ========== MODEL pot_ativa_inj_barramento24 ============
Epoch 1/20
606/614 [============================>.] - ETA: 0s - loss: 0.0685
Epoch 1: val_loss improved from inf to 0.01302, saving model to models-new/ann-pot_ativa_inj_barramento24.h5
614/614 [==============================] - 2s 3ms/step - loss: 0.0682 - val_loss: 0.0130
Epoch 2/20
593/614 [===========================>..] - ETA: 0s - loss: 0.0459
Epoch 2: val_loss improved from 0.01302 to 0.00820, saving model to models-new/ann-pot_ativa_inj_barramento24.h5
614/614 [==============================] - 2s 3ms/step - loss: 0.0456 - val_loss: 0.0082
Epoch 3/20
585/614 [===========================>..] - ETA: 0s - loss: 0.0407
Epoch 3: val_loss did not improve from 0.00820
614/614 [==============================] - 1s 2ms/step - loss: 0.0409 - val_loss: 0.0248
Epoch 4/20
603/614 [============================>.] - ETA: 0s - loss: 0.0387
Epoch 4: val_loss did not improve from 0.00820
614/614 [==============================] - 1s 2ms/s

In [21]:
cols_pred = ['pot_reativa_inj_barramento11']
model_id = 'pot_reativa_inj_barramento11'
pot_reativa_inj_barramento11_df = run(df, cols, cols_pred, model_id)

 ========== MODEL pot_reativa_inj_barramento11 ============
Epoch 1/20
610/614 [============================>.] - ETA: 0s - loss: 0.2023
Epoch 1: val_loss improved from inf to 0.19655, saving model to models-new/ann-pot_reativa_inj_barramento11.h5
614/614 [==============================] - 2s 3ms/step - loss: 0.2023 - val_loss: 0.1966
Epoch 2/20
606/614 [============================>.] - ETA: 0s - loss: 0.1286
Epoch 2: val_loss improved from 0.19655 to 0.09788, saving model to models-new/ann-pot_reativa_inj_barramento11.h5
614/614 [==============================] - 2s 3ms/step - loss: 0.1293 - val_loss: 0.0979
Epoch 3/20
606/614 [============================>.] - ETA: 0s - loss: 0.1089
Epoch 3: val_loss did not improve from 0.09788
614/614 [==============================] - 2s 3ms/step - loss: 0.1087 - val_loss: 0.1276
Epoch 4/20
606/614 [============================>.] - ETA: 0s - loss: 0.1030
Epoch 4: val_loss did not improve from 0.09788
614/614 [==============================] - 2s

In [22]:
cols_pred = ['pot_reativa_inj_barramento11']
model_id = 'pot_reativa_inj_barramento11'
pot_reativa_inj_barramento11_mae_df = run(df, cols, cols_pred, model_id)

 ========== MODEL pot_reativa_inj_barramento11 ============
Epoch 1/20
600/614 [============================>.] - ETA: 0s - loss: 0.2048
Epoch 1: val_loss improved from inf to 0.13581, saving model to models-new/ann-pot_reativa_inj_barramento11.h5
614/614 [==============================] - 2s 3ms/step - loss: 0.2047 - val_loss: 0.1358
Epoch 2/20
598/614 [============================>.] - ETA: 0s - loss: 0.1121
Epoch 2: val_loss improved from 0.13581 to 0.12272, saving model to models-new/ann-pot_reativa_inj_barramento11.h5
614/614 [==============================] - 2s 3ms/step - loss: 0.1119 - val_loss: 0.1227
Epoch 3/20
610/614 [============================>.] - ETA: 0s - loss: 0.0978
Epoch 3: val_loss did not improve from 0.12272
614/614 [==============================] - 2s 3ms/step - loss: 0.0975 - val_loss: 0.1228
Epoch 4/20
598/614 [============================>.] - ETA: 0s - loss: 0.0958
Epoch 4: val_loss did not improve from 0.12272
614/614 [==============================] - 2s

In [23]:
cols_pred = ['pot_reativa_inj_barramento12']
model_id = 'pot_reativa_inj_barramento12'
pot_reativa_inj_barramento12_mae_df = run(df, cols, cols_pred, model_id)

 ========== MODEL pot_reativa_inj_barramento12 ============
Epoch 1/20
612/614 [============================>.] - ETA: 0s - loss: 0.1855
Epoch 1: val_loss improved from inf to 0.10488, saving model to models-new/ann-pot_reativa_inj_barramento12.h5
614/614 [==============================] - 2s 3ms/step - loss: 0.1853 - val_loss: 0.1049
Epoch 2/20
602/614 [============================>.] - ETA: 0s - loss: 0.1194
Epoch 2: val_loss did not improve from 0.10488
614/614 [==============================] - 2s 3ms/step - loss: 0.1197 - val_loss: 0.1082
Epoch 3/20
605/614 [============================>.] - ETA: 0s - loss: 0.1025
Epoch 3: val_loss did not improve from 0.10488
614/614 [==============================] - 2s 3ms/step - loss: 0.1030 - val_loss: 0.1082
Epoch 4/20
604/614 [============================>.] - ETA: 0s - loss: 0.0893
Epoch 4: val_loss improved from 0.10488 to 0.08110, saving model to models-new/ann-pot_reativa_inj_barramento12.h5
614/614 [==============================] - 2s

In [34]:
cols_pred = ['pot_reativa_inj_barramento24']
model_id = 'pot_reativa_inj_barramento24'
pot_reativa_inj_barramento24_mae_df = run(df, cols, cols_pred, model_id)

 ========== MODEL pot_reativa_inj_barramento24 ============
Epoch 1/20
601/614 [============================>.] - ETA: 0s - loss: 0.2615
Epoch 1: val_loss improved from inf to 0.12107, saving model to models-new/ann-pot_reativa_inj_barramento24.h5
614/614 [==============================] - 1s 2ms/step - loss: 0.2598 - val_loss: 0.1211
Epoch 2/20
571/614 [==========================>...] - ETA: 0s - loss: 0.1734
Epoch 2: val_loss did not improve from 0.12107
614/614 [==============================] - 1s 1ms/step - loss: 0.1723 - val_loss: 0.1283
Epoch 3/20
592/614 [===========================>..] - ETA: 0s - loss: 0.1554
Epoch 3: val_loss improved from 0.12107 to 0.11244, saving model to models-new/ann-pot_reativa_inj_barramento24.h5
614/614 [==============================] - 1s 2ms/step - loss: 0.1555 - val_loss: 0.1124
Epoch 4/20
584/614 [===========================>..] - ETA: 0s - loss: 0.1465
Epoch 4: val_loss improved from 0.11244 to 0.08104, saving model to models-new/ann-pot_reati

# Without residual

In [27]:
cols_pred = ['pot_ativa_inj_barramento11']
model_id = 'pot_ativa_inj_barramento11_no_residual'
pot_reativa_inj_barramento11_mse_df = run(df, cols, cols_pred, model_id)

 ========== MODEL pot_ativa_inj_barramento11_no_residual ============
Epoch 1/20
604/614 [============================>.] - ETA: 0s - loss: 0.0891
Epoch 1: val_loss improved from inf to 0.00814, saving model to models-new/ann-pot_ativa_inj_barramento11_no_residual.h5
614/614 [==============================] - 2s 3ms/step - loss: 0.0885 - val_loss: 0.0081
Epoch 2/20
614/614 [==============================] - ETA: 0s - loss: 0.0498
Epoch 2: val_loss did not improve from 0.00814
614/614 [==============================] - 1s 2ms/step - loss: 0.0498 - val_loss: 0.0130
Epoch 3/20
600/614 [============================>.] - ETA: 0s - loss: 0.0481
Epoch 3: val_loss did not improve from 0.00814
614/614 [==============================] - 1s 2ms/step - loss: 0.0480 - val_loss: 0.0099
Epoch 4/20
610/614 [============================>.] - ETA: 0s - loss: 0.0453
Epoch 4: val_loss improved from 0.00814 to 0.00547, saving model to models-new/ann-pot_ativa_inj_barramento11_no_residual.h5
614/614 [======

In [29]:
cols_pred = ['pot_ativa_inj_barramento12']
model_id = 'pot_ativa_inj_barramento12_no_residual'
pot_reativa_inj_barramento11_mse_df = run(df, cols, cols_pred, model_id)

 ========== MODEL pot_ativa_inj_barramento12_no_residual ============
Epoch 1/20
606/614 [============================>.] - ETA: 0s - loss: 0.0850
Epoch 1: val_loss improved from inf to 0.01769, saving model to models-new/ann-pot_ativa_inj_barramento12_no_residual.h5
614/614 [==============================] - 2s 2ms/step - loss: 0.0848 - val_loss: 0.0177
Epoch 2/20
604/614 [============================>.] - ETA: 0s - loss: 0.0508
Epoch 2: val_loss improved from 0.01769 to 0.00485, saving model to models-new/ann-pot_ativa_inj_barramento12_no_residual.h5
614/614 [==============================] - 1s 2ms/step - loss: 0.0509 - val_loss: 0.0048
Epoch 3/20
584/614 [===========================>..] - ETA: 0s - loss: 0.0448
Epoch 3: val_loss did not improve from 0.00485
614/614 [==============================] - 1s 2ms/step - loss: 0.0447 - val_loss: 0.0108
Epoch 4/20
590/614 [===========================>..] - ETA: 0s - loss: 0.0421
Epoch 4: val_loss did not improve from 0.00485
614/614 [======

In [30]:
cols_pred = ['pot_ativa_inj_barramento24']
model_id = 'pot_ativa_inj_barramento24_no_residual'
run(df, cols, cols_pred, model_id)

 ========== MODEL pot_ativa_inj_barramento24_no_residual ============
Epoch 1/20
614/614 [==============================] - ETA: 0s - loss: 0.1073
Epoch 1: val_loss improved from inf to 0.01462, saving model to models-new/ann-pot_ativa_inj_barramento24_no_residual.h5
614/614 [==============================] - 1s 2ms/step - loss: 0.1073 - val_loss: 0.0146
Epoch 2/20
608/614 [============================>.] - ETA: 0s - loss: 0.0580
Epoch 2: val_loss did not improve from 0.01462
614/614 [==============================] - 1s 2ms/step - loss: 0.0581 - val_loss: 0.0201
Epoch 3/20
613/614 [============================>.] - ETA: 0s - loss: 0.0523
Epoch 3: val_loss improved from 0.01462 to 0.01147, saving model to models-new/ann-pot_ativa_inj_barramento24_no_residual.h5
614/614 [==============================] - 1s 2ms/step - loss: 0.0523 - val_loss: 0.0115
Epoch 4/20
603/614 [============================>.] - ETA: 0s - loss: 0.0476
Epoch 4: val_loss improved from 0.01147 to 0.00560, saving mod

,carga_subrede_138kv,pot_eolica_subrede_138kv,pot_solar_subrede_138kv,carga_subrede_230kv,pot_eolica_subrede_230kv,pot_solar_subrede_230kv,carga_subrede_138kv_pot_eolica_subrede_138kv,carga_subrede_138kv_pot_solar_subrede_138kv,carga_subrede_138kv_carga_subrede_230kv,carga_subrede_138kv_pot_eolica_subrede_230kv,...,pot_eolica_subrede_138kv_carga_subrede_230kv,pot_eolica_subrede_138kv_pot_eolica_subrede_230kv,pot_eolica_subrede_138kv_pot_solar_subrede_230kv,pot_solar_subrede_138kv_carga_subrede_230kv,pot_solar_subrede_138kv_pot_eolica_subrede_230kv,pot_solar_subrede_138kv_pot_solar_subrede_230kv,carga_subrede_230kv_pot_eolica_subrede_230kv,carga_subrede_230kv_pot_solar_subrede_230kv,pot_eolica_subrede_230kv_pot_solar_subrede_230kv,pot_ativa_inj_barramento24
0,1005.652996,359.159986,283.635482,1283.451268,8.607487,39.255017,361190.316109,285238.872660,1.290707e+06,8656.144796,...,460964.339560,3091.464805,14098.831457,364032.319496,2441.388643,11134.115759,11047.289728,50381.901710,337.887039,84.847859
1,617.833860,0.000000,87.547694,792.515072,0.000000,13.650222,0.000000,54089.929850,4.896426e+05,0.000000,...,0.000000,0.000000,0.000000,69382.867211,0.000000,1195.045419,0.000000,10818.006276,0.000000,86.258469
2,1141.981073,407.379918,258.826331,1332.578952,184.762495,256.556631,465220.155617,295574.771421,1.521780e+06,210995.272171,...,542865.903805,75268.529960,104516.019178,344906.521122,47821.398685,66403.611506,246210.611778,341881.966288,47402.043197,99.973801
3,868.235542,388.488818,437.258938,1251.923255,78.887444,215.605867,337299.799305,379643.750832,1.086964e+06,68492.882962,...,486358.185442,30646.889977,83760.468460,547414.632768,34494.240098,94275.592464,98761.026072,269921.999114,17008.595839,-13.398338
4,965.812828,340.724995,288.883583,1146.930810,9.182495,47.733434,329076.570789,279007.470097,1.107720e+06,8868.571411,...,390787.993959,3128.705540,16263.973951,331329.481395,2652.672037,13789.405350,10531.686357,54746.945788,438.312014,68.359353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,1047.049421,382.453688,145.038167,1320.391653,129.959999,22.976486,400447.913005,151862.128799,1.382515e+06,136074.541582,...,504988.657574,49703.680904,8787.441645,191507.184999,18849.160018,3332.467348,171598.097660,30337.959726,2986.024036,128.618697
8756,883.798108,345.058580,379.021118,1115.364948,154.097891,119.051114,304962.119740,334978.147292,9.857574e+05,136191.424584,...,384866.244783,53172.799420,41079.608253,422746.870145,58406.355030,45122.886331,171875.386346,132785.439482,18345.525582,16.233429
8757,927.840168,0.000000,147.900760,1000.380433,0.000000,46.292106,0.000000,137228.266198,9.281931e+05,0.000000,...,0.000000,0.000000,0.000000,147957.026485,0.000000,6846.637731,0.000000,46309.717488,0.000000,172.367601
8758,1117.080968,0.000000,74.904579,1103.291195,0.000000,204.838344,0.000000,83674.479764,1.232466e+06,0.000000,...,0.000000,0.000000,0.000000,82641.562680,0.000000,15343.329969,0.000000,225996.341616,0.000000,227.546462


In [31]:
cols_pred = ['pot_reativa_inj_barramento11']
model_id = 'pot_reativa_inj_barramento11_no_residual'
pot_reativa_inj_barramento12_mse_df = run(df, cols, cols_pred, model_id)

 ========== MODEL pot_reativa_inj_barramento11_no_residual ============
Epoch 1/20
596/614 [============================>.] - ETA: 0s - loss: 0.3662
Epoch 1: val_loss improved from inf to 0.22400, saving model to models-new/ann-pot_reativa_inj_barramento11_no_residual.h5
614/614 [==============================] - 2s 2ms/step - loss: 0.3668 - val_loss: 0.2240
Epoch 2/20
589/614 [===========================>..] - ETA: 0s - loss: 0.2650
Epoch 2: val_loss improved from 0.22400 to 0.20258, saving model to models-new/ann-pot_reativa_inj_barramento11_no_residual.h5
614/614 [==============================] - 1s 2ms/step - loss: 0.2652 - val_loss: 0.2026
Epoch 3/20
589/614 [===========================>..] - ETA: 0s - loss: 0.2423
Epoch 3: val_loss improved from 0.20258 to 0.18426, saving model to models-new/ann-pot_reativa_inj_barramento11_no_residual.h5
614/614 [==============================] - 1s 2ms/step - loss: 0.2431 - val_loss: 0.1843
Epoch 4/20
582/614 [===========================>..] -

In [28]:
cols_pred = ['pot_reativa_inj_barramento12']
model_id = 'pot_reativa_inj_barramento12_no_residual'
pot_reativa_inj_barramento12_mse_df = run(df, cols, cols_pred, model_id)

 ========== MODEL pot_reativa_inj_barramento12_no_residual ============
Epoch 1/20
595/614 [============================>.] - ETA: 0s - loss: 0.2291
Epoch 1: val_loss improved from inf to 0.16610, saving model to models-new/ann-pot_reativa_inj_barramento12_no_residual.h5
614/614 [==============================] - 2s 2ms/step - loss: 0.2289 - val_loss: 0.1661
Epoch 2/20
597/614 [============================>.] - ETA: 0s - loss: 0.1644
Epoch 2: val_loss improved from 0.16610 to 0.10952, saving model to models-new/ann-pot_reativa_inj_barramento12_no_residual.h5
614/614 [==============================] - 1s 2ms/step - loss: 0.1649 - val_loss: 0.1095
Epoch 3/20
591/614 [===========================>..] - ETA: 0s - loss: 0.1458
Epoch 3: val_loss did not improve from 0.10952
614/614 [==============================] - 1s 2ms/step - loss: 0.1460 - val_loss: 0.1126
Epoch 4/20
609/614 [============================>.] - ETA: 0s - loss: 0.1328
Epoch 4: val_loss improved from 0.10952 to 0.10302, savi

In [32]:
cols_pred = ['pot_reativa_inj_barramento24']
model_id = 'pot_reativa_inj_barramento24_no_residual'
pot_reativa_inj_barramento12_mse_df = run(df, cols, cols_pred, model_id)

 ========== MODEL pot_reativa_inj_barramento24_no_residual ============
Epoch 1/20
607/614 [============================>.] - ETA: 0s - loss: 0.2344
Epoch 1: val_loss improved from inf to 0.11330, saving model to models-new/ann-pot_reativa_inj_barramento24_no_residual.h5
614/614 [==============================] - 2s 2ms/step - loss: 0.2331 - val_loss: 0.1133
Epoch 2/20
607/614 [============================>.] - ETA: 0s - loss: 0.1616
Epoch 2: val_loss improved from 0.11330 to 0.10030, saving model to models-new/ann-pot_reativa_inj_barramento24_no_residual.h5
614/614 [==============================] - 1s 2ms/step - loss: 0.1615 - val_loss: 0.1003
Epoch 3/20
576/614 [===========================>..] - ETA: 0s - loss: 0.1442
Epoch 3: val_loss improved from 0.10030 to 0.09136, saving model to models-new/ann-pot_reativa_inj_barramento24_no_residual.h5
614/614 [==============================] - 1s 2ms/step - loss: 0.1453 - val_loss: 0.0914
Epoch 4/20
608/614 [============================>.] -